In [5]:
! ping spark-master

/usr/bin/sh: 1: ping: not found


In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("test_spark")
    .config("spark.driver.memory", "1g")
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/hive/")
    .config("spark.sql.catalogImplementation", "hive")
    # .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    # .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.executor.cores", 3)
    .config("spark.executor.memory", "6g")
    .config("spark.executor.instances", 2)
    .enableHiveSupport()
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/21 17:13:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
spark.sql(f"""
show tables
""").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [4]:
spark.sql(f"""
create database test
""").show()

++
||
++
++



In [5]:
spark.sql(f"""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
|     test|
+---------+



In [6]:
spark.sql(f"""
create table if not exists test.hive_table (key int, value string)
stored as orc
""").show()

++
||
++
++



24/07/21 17:14:15 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [8]:
spark.sql(f"""
insert into test.hive_table
values (1, "2")
""").show()

++
||
++
++



In [9]:
spark.sql(f"""
select * from test.hive_table
""").show()

+---+-----+
|key|value|
+---+-----+
|  1|    2|
+---+-----+

